In [12]:
dataset = "dataset02"

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict

import yaml

def load_class_names(yaml_path):
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)
    # Handle both list or dict formats
    if isinstance(data['names'], dict):
        class_names = [data['names'][i] for i in sorted(data['names'])]
    else:
        class_names = data['names']
    return class_names

IMAGE_DIR = f'{dataset}/val/images'
LABEL_DIR = f'{dataset}/val/labels'

YAML_PATH = f'{dataset}/data.yaml'  # or wherever your .yaml file is
CLASS_NAMES = load_class_names(YAML_PATH)
print(CLASS_NAMES)

In [ ]:
class_counts = defaultdict(int)
class_examples = defaultdict(list)

# Parse annotations and collect counts + examples
for label_file in os.listdir(LABEL_DIR):
    if not label_file.endswith('.txt'):
        continue
    label_path = os.path.join(LABEL_DIR, label_file)
    image_name = os.path.splitext(label_file)[0] + '.jpg'
    image_path = os.path.join(IMAGE_DIR, image_name)

    if not os.path.exists(image_path):
        continue

    with open(label_path, 'r') as f:
        for line in f:
            cls_id = int(line.strip().split()[0])
            class_counts[cls_id] += 1
            if len(class_examples[cls_id]) < 3:
                class_examples[cls_id].append((image_path, line.strip()))

total_cones = 0
for c in class_counts: 
    total_cones += class_counts[c] 

for i, cls in enumerate(CLASS_NAMES):
    print(f"Class {i} - {cls}: {class_counts[i]} occurences ({round(class_counts[i]/total_cones, 2)*100}%)")

print(f"Total: {total_cones}")

In [ ]:
# Show 1–3 examples for each class
for cls_id, examples in class_examples.items():
    print(f"Class {cls_id} ({CLASS_NAMES[cls_id]}): {class_counts[cls_id]} occurrences")
    for img_path, annotation in examples:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        _, x, y, bw, bh = map(float, annotation.split())
        x1 = int((x - bw / 2) * w)
        y1 = int((y - bh / 2) * h)
        x2 = int((x + bw / 2) * w)
        y2 = int((y + bh / 2) * h)
        cv2.rectangle(img, (x1, y1), (x2, y2), (255,0,0), 2)
        cv2.putText(img, CLASS_NAMES[cls_id], (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1)
        plt.imshow(img)
        plt.title(f"Class {cls_id} Example")
        plt.axis('off')
        plt.show()


In [ ]:
import torch 

torch.cuda.empty_cache()

from ultralytics import YOLO

model = YOLO("yolov10n.yaml")

# Train with small batch size and limited workers
results = model.train(
    data='dataset02/data.yaml',
    epochs=100,
    imgsz=2000,    
    batch=2,
    workers=0,         # Avoid multiprocessing overhead
    device=0           # Make sure it uses the GPU
)

model.export()

In [ ]:
metrics = model.val(data="dataset02/data.yaml", split='test')  # uses test images

# Optional: print metrics
print(metrics.box.map)      # mAP@0.5:0.95
print(metrics.box.map50)    # mAP@0.5

In [ ]:
print(metrics.box.p)
print(metrics.box.r)